In [27]:
import sys
print(sys.path)
sys.path.insert(0, '/workspace/')
import numpy as np
from spacy import prefer_gpu
import torch
from spacy.util import minibatch, compounding
from spacy.training import offsets_to_biluo_tags
import json
import spacy
import pandas as pd
from spacy.training import Example
import random
import warnings
from qrytool import load_data_into_dataframe, insert_dataframe_into_table




if prefer_gpu():
    print("GPU is being used.")
else:
    print("CPU is being used.")
# 새 모델 생성 또는 기존 모델 로드
nlp = spacy.blank('ko')  # 새 모델을 생성하거나
# NER 파이프라인 추가
if 'ner' not in nlp.pipe_names:
    ner = nlp.add_pipe('ner', last=True)
else:
    ner = nlp.get_pipe('ner')

# 저장된 모델 로드
nlp = spacy.load("/workspace/ko_addr_ner_model_999/")

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_colwidth', None)

['/workspace/', '/', '/opt/conda/envs/py39/lib/python39.zip', '/opt/conda/envs/py39/lib/python3.9', '/opt/conda/envs/py39/lib/python3.9/lib-dynload', '', '/opt/conda/envs/py39/lib/python3.9/site-packages', '/opt/conda/envs/py39/lib/python3.9/site-packages/shap-0.42.1-py3.9-linux-x86_64.egg', '/opt/conda/envs/py39/lib/python3.9/site-packages/slicer-0.0.7-py3.9.egg']
GPU is being used.


사전에 naver_complexes와 addressalls 의 매핑을 만들어 두어 customatrix의 좌표 및 주소로 naver_complexes의 후보 데이터들을 빠르게 가져올 수 있도록 준비한다.

```
CREATE MATERIALIZED VIEW address_with_extracted AS
SELECT old_addr,new_addr,building_name,sub_building_no,lati,longi,hcode,r1,r2,r3,bcode, substring(old_addr FROM '[동가리] ([0-9]+[-]*[0-9]*)' ) AS extracted_number
FROM addressalls;

CREATE INDEX idx_address_with_extracted ON address_with_extracted(extracted_number);

CREATE TABLE alladdr_naver_joins AS
SELECT a.*, n.*
FROM address_with_extracted a
JOIN naver_complexes n
ON n.detail_address LIKE '%' || a.extracted_number
AND 6371000 * 2 * ASIN(SQRT(
    POWER(SIN((RADIANS(n.latitude) - RADIANS(a.lati)) / 2), 2) +
    COS(RADIANS(a.lati)) * COS(RADIANS(n.latitude)) *
    POWER(SIN((RADIANS(n.longitude) - RADIANS(a.longi)) / 2), 2)
)) <= 3000;
```


In [69]:
def get_address_by_name_mdn(name,mdn):
    qry_name_mdn=f"SELECT all_id,이름,결제전화,회원주소,회원주소1,회원lati,회원longi,가입일시 FROM customatrix WHERE 이름='{name}' and 결제전화 like '%{mdn}'"
    df = load_data_into_dataframe(qry_name_mdn)
    return df

def get_address_by_cid(cid):
    qry_cid=f"SELECT all_id,이름,결제전화,회원주소,회원주소1,회원lati,회원longi,가입일시 FROM customatrix WHERE all_id={cid}"
    df = load_data_into_dataframe(qry_cid)
    return df

def get_address_by_addr(addr):
    qry_cid=f"SELECT all_id,이름,결제전화,회원주소,회원주소1,회원lati,회원longi,가입일시 FROM customatrix WHERE 회원주소={addr}"
    df = load_data_into_dataframe(qry_cid)
    return df

# def is_complex(address, lat,lon):

def get_building_dong_ho(address, lat, lon):
    doc = nlp(address.strip())
    # Create a dictionary to store the entities for this address
    entities = {'address': address.strip()}
    for ent in doc.ents:
        entities[ent.label_] = ent.text
    second_area = address.split()[1]
    apt_name = entities.get('건물') or entities.get('단지')
    dong_name=entities.get('건물동')
    ho_name=entities.get('건물호')
    return apt_name, dong_name, ho_name

def get_address_info_name_mdn(name, mdn):
    df=get_address_by_name_mdn(name,mdn)
    if df.empty:
        return None
    qry_alladdr_namver_joins = f"SELECT * FROM alladdr_naver_joins WHERE lati={df['회원lati'].iloc[0]} AND longi={df['회원longi'].iloc[0]}"
    df_all = load_data_into_dataframe(qry_alladdr_namver_joins)
    addr= df['회원주소'].iloc[0]
    if len(df_all) == 0:
        return addr, None
    elif len(df_all) == 1:
        apt_name, dong_name, ho_name=get_building_dong_ho(addr, df_all['lati'], df_all['longi'], )
        display(df_all)
        return {'주소':addr,'아파트명':apt_name, '동':dong_name, '호':ho_name, '건물명':df_all['complex_name'].iloc[0],'건물#':df_all['complex_no'].iloc[0], }
    else:
        return addr, df_all


def search_complex_realprice(complex_no,dong_name, ho_name):
    qry="""
        SELECT
            nc.complex_name,
            ncp.complex_no,
            ncd.build_name,
            ncd.ho_name,
            ncd.ho_floor,
            ncp.*
        FROM naver_complexes  nc
        LEFT JOIN naver_complex_dongho ncd ON nc.complex_no = ncd.complex_no
        LEFT JOIN naver_complex_realprices ncp ON ncp.complex_no = nc.complex_no and ncp.pyeong_no = ncd.pyeong_no
        WHERE
    """
    dong_name=dong_name.replace("동","") if dong_name else ''
    ho_name=ho_name.replace("호","") if ho_name else ''
    target_qry=qry+f"nc.complex_no='{complex_no}'"
    if dong_name and dong_name.strip()!='':
        target_qry+=f"and build_name='{dong_name}'"
    else:
        target_qry+="and build_name='1'"

    if ho_name and ho_name.strip()!='':
        target_qry+=f"and ho_name='{ho_name}'"
    # print(target_qry)
    df = load_data_into_dataframe(target_qry)
    ordered_df = df.sort_values(by=['trade_base_year', 'trade_base_month'], ascending=False)
    # display(df) #dong ho pyeong and price
    return ordered_df


In [ ]:
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, Output
from IPython.display import clear_output
import warnings
warnings.simplefilter("ignore")

def display_with_style(name, price):
    # Create a DataFrame
    df = pd.DataFrame({'Name': [name], 'Price': [price]})

    # Define a function to color the 'Name' cell grey
    def color_name_cell(val):
        color = 'grey' if val == name else 'white'
        return 'background-color: %s' % color

    # Apply the style to the DataFrame and display it
    styled_df = df.style.applymap(color_name_cell)

    # Hide the index and headers
    styled_df.hide_index().hide_columns()

    # Display the styled DataFrame
    display(styled_df)
# Create text input fields
name_input = widgets.Text(
    value='',
    placeholder='Enter name',
    description='Name:',
)

mdn_input = widgets.Text(
    value='',
    placeholder='Enter last 4 digits of phone number',
    description='MDN:',
)

# Create a button
button = widgets.Button(description='Find')

# Display the widgets
output = Output()

def on_button_clicked(b):
  # Optionally clear previous output
  # clear_output(wait=True)
  output.clear_output(wait=True)
  # Get the input values
  name = name_input.value
  mdn = mdn_input.value

  # Call the function with the input values
  result = get_address_info_name_mdn(name, mdn)
  with output:
    if result is None:
      display('User not found')
    else:
      print(result)
      df_price=search_complex_realprice(result['건물#'], result['동'], result['호'])
      # Print the result
      display_with_style(name, df_price['formatted_price'].iloc[0])
      display(pd.DataFrame([result]))
      display(df_price)

# Set the function to be called when the button is clicked
button.on_click(on_button_clicked)
display(name_input, mdn_input, button, output)
# Use interact to display widgets
# display(interact(on_button_clicked, name=name_input, mdn=mdn_input))


In [ ]:
from IPython.display import clear_output

def on_button_clicked(b):
    # Clear the output
    clear_output(wait=True)

    # Get the input values
    name = name_input.value
    mdn = mdn_input.value

    # Call the function with the input values
    result = get_address_info_name_mdn(name, mdn)
    if result is None:
        display('User not found')
    else:
        df_price=search_complex_price(result['건물#'], result['동'], result['호'])
        # Print the result
        display(pd.DataFrame([result]))
        display(df_price)